# Embedding word2vec fasttext

## Импорты

In [1]:
import re
from tqdm.notebook import tqdm
import string
import nltk
import numpy as np
import pandas as pd
import annoy

from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words

from multiprocessing import Pool
from gensim.models import Word2Vec, FastText

## Настройки

In [2]:
DATA_PATH = './data/lenta-ru-news.csv.gz'
NUM_WORKERS = 32
TAKE_TOP_N = 50000

## Загрузка и обработка данных

In [3]:
data = pd.read_csv(DATA_PATH, compression='gzip')
print(len(data))
data.head()

739351


,url,title,text,topic,tags
0,https://lenta.ru/news/2018/12/14/cancer/,Названы регионы России с самой высокой смертно...,Вице-премьер по социальным вопросам Татьяна Го...,Россия,Общество
1,https://lenta.ru/news/2018/12/15/doping/,Австрия не представила доказательств вины росс...,Австрийские правоохранительные органы не предс...,Спорт,Зимние виды
2,https://lenta.ru/news/2018/12/15/disneyland/,Обнаружено самое счастливое место на планете,Сотрудники социальной сети Instagram проанализ...,Путешествия,Мир
3,https://lenta.ru/news/2018/12/15/usa25/,В США раскрыли сумму расходов на расследование...,С начала расследования российского вмешательст...,Мир,Политика
4,https://lenta.ru/news/2018/12/15/integrity/,Хакеры рассказали о планах Великобритании зами...,Хакерская группировка Anonymous опубликовала н...,Мир,Общество


In [4]:
print(data['text'][1])

Австрийские правоохранительные органы не представили доказательств нарушения российскими биатлонистами антидопинговых правил. Об этом сообщил посол России в Вене Дмитрий Любинский по итогам встречи уполномоченного адвоката дипмиссии с представителями прокуратуры страны, передает ТАСС. «Действует презумпция невиновности. Каких-либо ограничений свободы передвижения для команды нет», — добавили в посольстве. Международный союз биатлонистов (IBU) также не будет применять санкции к российским биатлонистам. Все они продолжат выступление на Кубке мира. Полиция нагрянула в отель сборной России в Хохфильцене вечером 12 декабря. Как написал биатлонист Александр Логинов, их считают виновными в махинациях с переливанием крови. Биатлонисту Антону Шипулину, также попавшему в список, полиция нанесла отдельный визит: сейчас он тренируется отдельно в австрийском Обертиллахе. Обвинения спортсмен назвал бредом, а также указал на «охоту на ведьм» в мировом биатлоне. В Австрии прием допинга — уголовное пре

In [5]:
data = data[:TAKE_TOP_N]
data['text'] = data['text'].astype(np.str_)

### Этот блок кода вынесен в файл и импортирован т.к. по другому многопоточка в юпитере на windows работать не хочет

In [6]:
morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)

def preprocess_data(text):
    text = text.lower()
    words = text.split()
    words = [word for word in words if word not in sw | exclude]
    words = list(map(str.strip, words))
    words = [word for word in words if len(word) > 2]
    words = [morpher.parse(word)[0].normal_form for word in words]
    return words

preprocess_data(data['text'][0])[:5]

['вице-премьер', 'социальный', 'вопрос', 'татьяна', 'голиков']

In [7]:
%%time
sentences = []

if __name__ ==  '__main__':
    from preprocess import preprocess_data # Та-же функция, что описана выше
    N = len(data)

    with Pool(NUM_WORKERS) as pool:
        sentences = list(tqdm(
            pool.imap(preprocess_data, data['text'][:N]), total=N
        ))

  0%|          | 0/50000 [00:00<?, ?it/s]

Wall time: 30 s


In [8]:
print(len(sentences))
sentences = [i for i in sentences if len(i) > 5]
print(len(sentences))

50000
50000


## Модели
### Word2Vec

In [9]:
%%time
model_w2v = Word2Vec(sentences=sentences, vector_size=300, window=5, min_count=3, workers=NUM_WORKERS)

Wall time: 16.5 s


### FastText

In [10]:
%%time
model_ft = FastText(sentences=sentences, vector_size=300, window=5, min_count=3, workers=NUM_WORKERS)

Wall time: 1min 48s


In [11]:
%%time
w2v_index = annoy.AnnoyIndex(300, 'angular')
ft_index = annoy.AnnoyIndex(300, 'angular')

counter = 0
index_map = {}
for text in tqdm(data['text']):
    n_w2v, n_ft = 0, 0
    vector_w2v = np.zeros(300)
    vector_ft = np.zeros(300)

    for word in preprocess_data(text):
        if word in model_w2v.wv:
            vector_w2v += model_w2v.wv[word]
            n_w2v += 1
        
        if word in model_ft.wv:
            vector_ft += model_ft.wv[word]
            n_ft += 1
    
    if n_w2v > 0:
        vector_w2v = vector_w2v / n_w2v
    
    if n_ft > 0:
        vector_ft = vector_ft / n_ft

    index_map[counter] = text
    w2v_index.add_item(counter, vector_w2v)
    ft_index.add_item(counter, vector_ft)
    
    counter += 1

w2v_index.build(10)
ft_index.build(10)

  0%|          | 0/50000 [00:00<?, ?it/s]

Wall time: 7min 17s


True

In [12]:
def get_response(question, index, model, index_map):
    vector = np.zeros(300)
    norm = 0

    for word in preprocess_data(question):
        if word in model.wv:
            vector += model.wv[word]
            norm += 1

    if norm > 0:
        vector = vector / norm

    answers = index.get_nns_by_vector(vector, 5, )
    return [index_map[i] for i in answers]

## Проверка результатов
### Искуственный интелект

In [13]:
TEXT = "Искуственный интелект"
print(*get_response(TEXT, w2v_index, model_w2v, index_map), sep='\n\n')

Россия сохраняет готовность к проведению встречи президента России Владимира Путина с американским коллегой Дональдом Трампом, заявил официальный представитель Кремля Дмитрий Песков. Его цитирует ТАСС в пятницу, 14 декабря. «Мы по-прежнему убеждены, что эта встреча в равной степени необходима как для Москвы, так и для Вашингтона», — сказал пресс-секретарь российского лидера. По его словам, у сторон есть срочные вопросы для обсуждения, в частности, касающиеся «стратегической безопасности всего мира». 13 декабря советник американского президента по национальной безопасности Джон Болтон заявил, что встреча президентов не состоится до тех пор, пока Россия не отпустит задержанных после инцидента в Керченском проливе украинских моряков. Переговоры двух лидеров готовились в рамках саммита G20 в Аргентине, однако Трамп отменил встречу. В Москве полагают, что реальные причины такого решения кроются во внутриполитической повестке. Инцидент в Керченском проливе с участием трех украинских военных 

In [14]:
TEXT = "Искуственный интелект"
print(*get_response(TEXT, ft_index, model_ft, index_map), sep='\n\n')

Один из создателей российского сегмента интернета Валерий Бардин скончался в Москве. Об этом сообщает ТАСС. О смерти Бардина сообщила его супруга. Причиной кончины программиста стало онкологическое заболевание. Бардин был сотрудником в Институте атомной энергии имени И.В. Курчатова, где в конце 1980-х была создана компьютерная сеть. Позднее ее подключили к мировому сегменту. Кроме того, программист был одним из активных участников множества проектов, связанных с компьютерными сетями и информационными технологиями. В частности, он — один из создателей советской операционной системы «ДЕМОС», а также учредитель первого сетевого издания «Национальная служба новостей». Как отмечает ТАСС, Бардин также создал для информационного агентства систему для определения тематики материалов с применением искусственного интеллекта.

Системы искусственного интеллекта будут способны заменить человека через 10-15 лет, когда произойдёт замена оператора по обработке данных на робота. Об этом РИА Новости зая

### Космос

In [15]:
TEXT = "Космос"
print(*get_response(TEXT, w2v_index, model_w2v, index_map), sep='\n\n')

Нидерландский астроном Марко Лангбрук ночью 7 февраля 2018 сфотографировал в небе над Лейденом советские космические аппараты «Космос-1860» и «Космос-1412», каждый из которых до отработки задач переносил по ядерному реактору. В комментарии под своим твитом эксперт отметил, что речь идет о бортовой ядерной энергетической установке БЭС-5 «Бук» мощностью 5 киловатт.  Запущенные в 1980-х годах космические аппараты серии УС-А (Управляемый Спутник Активный), к которым относятся «Космос-1860» и «Космос-1412», входили в состав системы морской космической разведки и целеуказания МКРЦ «Легенда». Из 33 спутников УС-А 31 оснащались БЭС-5 «Бук», а два переносили ТЭУ-5 «Тополь». Типовой такой спутник имел цилиндрическую форму (диаметр — 1,3 метра, длина — 10 метров), на одном конце космического аппарата располагался радар, а на другом — ядерная энергетическая установка. Из общей массы спутника 3800 килограммов на ядерную установку (реактор на быстрых нейтронах и термоэлектрический генератор в случае

In [16]:
TEXT = "Космос"
print(*get_response(TEXT, ft_index, model_ft, index_map), sep='\n\n')

Ракета-носитель «Союз-ФГ» с транспортным пилотируемым кораблем «Союз МС-11» впервые стартовала после аварии. Об этом сообщает «Роскосмос». Пуск произошел 3 декабря в 14:31 по московскому времени со стартового комплекса площадки № 1 «Гагаринский старт» космодрома Байконур. В состав основного экипажа экспедиции МКС-58/59 входят космонавт Олег Кононенко, астронавты Давид Сен-Жак и Энн Макклейн. Стыковка «Союза МС-11» с МКС и причаливание должны состояться через шесть часов (по «короткой» схеме). Во время экспедиции 58/59 экипажу МКС предстоит, в частности, выход в открытый космос, а также работа с грузовыми кораблями, обслуживание бортовых систем станции, проведение бортовых фото- и видеосъемок. В ходе полета «Союза-ФГ» с «Союзом МС-10», стартовавшего 11 октября с космодрома Байконур к МКС, на 123-й секунде произошло аварийное отключение двигателей второй ступени ракеты. Находившиеся в корабле космонавт «Роскосмоса» Алексей Овчинин и астронавт НАСА Тайлер Николас Хейг задействовали систем

### Политика

In [17]:
TEXT = "Политика"
print(*get_response(TEXT, w2v_index, model_w2v, index_map), sep='\n\n')

Министр иностранных дел Германии Хайко Маас призвал страны Евросоюза сплотиться перед лицом внешних вызовов, в числе которых назвал Россию и США. Его слова передает ТАСС в среду, 13 июня. В частности, глава ведомства назвал угрозой ЕС «эгоистичную политику президента Соединенных Штатов Дональда Трампа "Америка прежде всего"». В качестве еще одного вызова он упомянул Россию, которая «атакует международное право и суверенитет государств». «Миропорядка, который мы знали и к которому привыкли, больше не существует. (...) Я убежден, что нам нужно проявить больше мужества. Наше малодушие питает национализм и изоляцию», — добавил он. До этого Маас призывал страны запада усилить давление на Россию в связи с событиями в Сирии. В конце мая австрийский канцлер Себастьян Курц также заявлял, что Европа может все меньше полагаться на Вашингтон, и призвал Евросоюз оставаться единым.

Министр иностранных дел Германии Зигмар Габриэль призвал пересмотреть европейскую и немецкую политику в отношении США 

In [18]:
TEXT = "Политика"
print(*get_response(TEXT, ft_index, model_ft, index_map), sep='\n\n')

Министр обороны Германии Урсула фон дер Ляйен призвала польскую молодежь продолжать свое сопротивление властям. Об этом она заявила в телеэфире, ее слова передает The Local. Женщина рассказывала о борьбе Польши и стран Балтии во времена власти коммунистов. Она также отметила, что и сегодня этот регион не должен оставаться в арьергарде происходящих в ЕС изменений. «Здоровое демократическое сопротивление молодого поколения в Польше должно быть поддержано», — сказала фон дер Ляйен. Власти Польши возмутились и вызвали военного атташе Германии, который объяснил, что слова министра вырвали из контекста, однако Варшава таким пояснением не удовлетворилась. «Польское министерство обороны считает недопустимым, что министр страны НАТО призывает граждан другого государства к антиправительственным действиям», — отметила представитель оборонного ведомства Польши. Министр иностранных дел Польской Республики Витольд Ващиковский заявил, что Варшава не собирается начинать большую дискуссию из-за этого д

Модели тестировались на темах "Искуственный интелект", Космос и Политика.<br>
Word2Vec не смог справиться с запросом на тему "Искуственный интелект". С остальными темами она показала неплохой результат.<br>
FastText смог выдать релевантный ответ на все темы.